In [1]:
import boto3
from botocore.exceptions import ClientError

print("✅ Imports loaded.")


✅ Imports loaded.


In [2]:
ACCESS_KEY = "user_37sKcYrvnk9UXaIY3B3Zr90MH0g"
SECRET_KEY = "rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n"

BUCKET = "e9tcw5eupu"
ENDPOINT_URL = "https://s3api-eu-ro-1.runpod.io"
REGION = "eu-ro-1"

print("✅ Settings loaded.")
print("Bucket:", BUCKET)
print("Endpoint:", ENDPOINT_URL)
print("Region:", REGION)


✅ Settings loaded.
Bucket: e9tcw5eupu
Endpoint: https://s3api-eu-ro-1.runpod.io
Region: eu-ro-1


In [3]:
s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT_URL,
    region_name=REGION,
)

print("✅ S3 client created. Testing access...")

try:
    r = s3.list_objects_v2(Bucket=BUCKET, MaxKeys=3)
    print("✅ Connected. KeyCount:", r.get("KeyCount", 0))
except ClientError as e:
    print("❌ Connection failed:")
    print(e)
    raise


✅ S3 client created. Testing access...
✅ Connected. KeyCount: 3


In [4]:
prefix = "tight loose clothing model new/data/lower body clothing categories/"
print("🔎 Listing folders under:", prefix)

folders = set()
token = None

try:
    while True:
        kwargs = {
            "Bucket": BUCKET,
            "Prefix": prefix,
            "Delimiter": "/",   # makes S3 return "folders" in CommonPrefixes
            "MaxKeys": 1000
        }
        if token:
            kwargs["ContinuationToken"] = token

        resp = s3.list_objects_v2(**kwargs)

        for cp in resp.get("CommonPrefixes", []):
            full_prefix = cp["Prefix"]  # e.g. prefix + "jeans/"
            name = full_prefix[len(prefix):].strip("/")
            if name:
                folders.add(name)

        if resp.get("IsTruncated"):
            token = resp.get("NextContinuationToken")
            print("…more results, continuing")
        else:
            break

    folders = sorted(folders)
    print(f"✅ Found {len(folders)} folder(s):")
    for i, f in enumerate(folders, start=1):
        print(f"{i}. {f}")

except ClientError as e:
    print("❌ Error listing folders:")
    print(e)
    raise


🔎 Listing folders under: tight loose clothing model new/data/lower body clothing categories/
✅ Found 5 folder(s):
1. cargo pants
2. culottes
3. shorts
4. skirt
5. trousers


### Data Augementation

In [5]:
# Cell 1: Import required libraries
import os
import boto3
from botocore.exceptions import ClientError
import albumentations as A
import cv2
import numpy as np
from pathlib import Path
import io

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [6]:
# Cell 2: Define RunPod S3 credentials and paths
ACCESS_KEY = "user_37sKcYrvnk9UXaIY3B3Zr90MH0g"
SECRET_KEY = "rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n"
BUCKET = "e9tcw5eupu"
ENDPOINT_URL = "https://s3api-eu-ro-1.runpod.io"
REGION = "eu-ro-1"

# S3 paths
SOURCE_PREFIX = "tight loose clothing model new/data/lower body clothing categories/"
TARGET_PREFIX = "tight loose clothing model new augmented/data/lower body clothing categories/"

print("✅ Credentials and paths configured")
print(f"📂 Source: {SOURCE_PREFIX}")
print(f"📂 Target: {TARGET_PREFIX}")

✅ Credentials and paths configured
📂 Source: tight loose clothing model new/data/lower body clothing categories/
📂 Target: tight loose clothing model new augmented/data/lower body clothing categories/


In [7]:
# Cell 3: Initialize S3 client and test connection
print("🔌 Connecting to RunPod S3...")
s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT_URL,
    region_name=REGION,
)

# Test connection
try:
    r = s3.list_objects_v2(Bucket=BUCKET, MaxKeys=3)
    print(f"✅ Connected successfully. KeyCount: {r.get('KeyCount', 0)}")
except ClientError as e:
    print(f"❌ Connection failed: {e}")
    raise

🔌 Connecting to RunPod S3...
✅ Connected successfully. KeyCount: 3


In [8]:
# Cell 4: Define augmentation pipeline
print("🎨 Setting up augmentation pipeline...")

# Create augmentation pipeline with all transformations
augmentation_pipeline = A.Compose([
    # Apply random brightness and contrast changes
    A.RandomBrightnessContrast(
        brightness_limit=0.1,  # Limit brightness change to ±10%
        contrast_limit=0.1,    # Limit contrast change to ±10%
        p=0.8                  # Apply this augmentation 80% of the time
    ),
    # Apply random hue, saturation, and value changes
    A.HueSaturationValue(
        hue_shift_limit=5,     # Shift hue by ±5 degrees
        sat_shift_limit=10,    # Shift saturation by ±10
        val_shift_limit=5,     # Shift value/brightness by ±5
        p=0.6                  # Apply this augmentation 60% of the time
    ),
    # Add random Gaussian noise to the image
    A.GaussNoise(
        var_limit=(5.0, 15.0), # Noise variance between 5 and 15
        p=0.5                  # Apply this augmentation 50% of the time
    ),
    # Apply random Gaussian blur
    A.GaussianBlur(
        blur_limit=(3, 5),     # Kernel size between 3 and 5
        p=0.4                  # Apply this augmentation 40% of the time
    )
])

print("✅ Augmentation pipeline ready with 4 transformations")

🎨 Setting up augmentation pipeline...
✅ Augmentation pipeline ready with 4 transformations


/tmp/ipykernel_2060/3576183489.py:20: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(


In [9]:
# Cell 5: List all folders in source directory
print(f"📂 Listing folders in: {SOURCE_PREFIX}")

# Query S3 for folders (using Delimiter to get only direct subfolders)
response = s3.list_objects_v2(Bucket=BUCKET, Prefix=SOURCE_PREFIX, Delimiter='/')

# Initialize empty list to store folder names
folders = []

# Extract folder names from common prefixes
if 'CommonPrefixes' in response:
    for prefix in response['CommonPrefixes']:
        folder_path = prefix['Prefix']  # Get full folder path
        folder_name = folder_path.rstrip('/').split('/')[-1]  # Extract just folder name
        folders.append(folder_name)  # Add to list

# Display found folders
print(f"✅ Found {len(folders)} folders:")
for idx, folder in enumerate(folders, 1):
    print(f"   {idx}. {folder}")

📂 Listing folders in: tight loose clothing model new/data/lower body clothing categories/
✅ Found 5 folders:
   1. cargo pants
   2. culottes
   3. shorts
   4. skirt
   5. trousers


In [10]:
# Cell 6: Configure augmentation parameters
# Target number of images for each folder
TARGET_IMAGES_PER_FOLDER = 200

# Original number of images in each folder
ORIGINAL_IMAGES_PER_FOLDER = 20

# Calculate how many augmented versions needed per original image
AUGMENTATIONS_PER_IMAGE = TARGET_IMAGES_PER_FOLDER // ORIGINAL_IMAGES_PER_FOLDER

# Display configuration
print(f"📊 Augmentation Configuration:")
print(f"   - Original images per folder: {ORIGINAL_IMAGES_PER_FOLDER}")
print(f"   - Target images per folder: {TARGET_IMAGES_PER_FOLDER}")
print(f"   - Augmentations per original image: {AUGMENTATIONS_PER_IMAGE}")
print(f"   - Total folders: {len(folders)}")
print(f"   - Expected total images: {len(folders) * TARGET_IMAGES_PER_FOLDER}")

📊 Augmentation Configuration:
   - Original images per folder: 20
   - Target images per folder: 200
   - Augmentations per original image: 10
   - Total folders: 5
   - Expected total images: 1000


In [11]:
# Cell 7: Main augmentation loop
# Initialize counter for total images created across all folders
total_images_created = 0

# Loop through each folder
for folder_idx, folder_name in enumerate(folders, 1):
    print(f"\n{'='*60}")
    print(f"📁 Processing folder {folder_idx}/{len(folders)}: {folder_name}")
    print(f"{'='*60}")
    
    # Define source and target paths for current folder
    folder_source_prefix = f"{SOURCE_PREFIX}{folder_name}/"
    folder_target_prefix = f"{TARGET_PREFIX}{folder_name}/"
    
    # List all objects in current source folder
    print(f"🔍 Listing images in {folder_name}...")
    response = s3.list_objects_v2(Bucket=BUCKET, Prefix=folder_source_prefix)
    
    # Initialize list to store image keys (S3 paths)
    image_keys = []
    
    # Filter for image files only
    if 'Contents' in response:
        for obj in response['Contents']:
            key = obj['Key']  # Get S3 object key (path)
            # Skip if it's a folder marker
            if key.endswith('/'):
                continue
            # Check if file has image extension
            if key.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                image_keys.append(key)  # Add to list of image keys
    
    print(f"✅ Found {len(image_keys)} images in {folder_name}\n")
    
    # Initialize counter for images created in current folder
    folder_images_created = 0
    
    # Process each original image in the folder
    for img_idx, image_key in enumerate(image_keys, 1):
        # Extract filename from full S3 path
        original_filename = image_key.split('/')[-1]
        # Split filename into name and extension
        original_name, original_ext = os.path.splitext(original_filename)
        
        print(f"  🖼️  Image {img_idx}/{len(image_keys)}: {original_filename}")
        
        # Download image from S3
        print(f"     ⬇️  Downloading from S3...")
        img_obj = s3.get_object(Bucket=BUCKET, Key=image_key)
        img_data = img_obj['Body'].read()  # Read image bytes
        
        # Convert bytes to numpy array
        img_array = np.frombuffer(img_data, np.uint8)
        # Decode array into OpenCV image format
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        
        # Check if image was decoded successfully
        if image is None:
            print(f"     ❌ Failed to decode image, skipping...")
            continue
        
        print(f"     ✅ Downloaded (shape: {image.shape})")
        
        # Create multiple augmented versions of current image
        print(f"     🔄 Creating {AUGMENTATIONS_PER_IMAGE} augmented versions...")
        for aug_idx in range(1, AUGMENTATIONS_PER_IMAGE + 1):
            # Apply augmentation pipeline to image
            augmented = augmentation_pipeline(image=image)
            augmented_image = augmented['image']  # Get augmented image
            
            # Create new filename with augmentation index
            new_filename = f"{original_name}_aug{aug_idx}{original_ext}"
            # Create full S3 key (path) for new image
            new_key = f"{folder_target_prefix}{new_filename}"
            
            # Encode augmented image back to bytes
            _, buffer = cv2.imencode(original_ext, augmented_image)
            img_bytes = io.BytesIO(buffer.tobytes())
            
            # Upload augmented image to S3
            s3.put_object(
                Bucket=BUCKET,
                Key=new_key,
                Body=img_bytes.getvalue(),
                ContentType='image/jpeg' if original_ext.lower() in ['.jpg', '.jpeg'] else 'image/png'
            )
            
            # Increment counters
            folder_images_created += 1
            total_images_created += 1
            
            # Print progress every 20 images or at the end
            if aug_idx % 20 == 0 or aug_idx == AUGMENTATIONS_PER_IMAGE:
                print(f"        ✔️  Created {aug_idx}/{AUGMENTATIONS_PER_IMAGE} augmentations")
        
        print(f"     ✅ Completed augmentation for {original_filename}\n")
    
    # Print summary for current folder
    print(f"✅ Folder '{folder_name}' complete: {folder_images_created} images created")
    print(f"📊 Total progress: {total_images_created}/{len(folders) * TARGET_IMAGES_PER_FOLDER} images")


📁 Processing folder 1/5: cargo pants
🔍 Listing images in cargo pants...
✅ Found 20 images in cargo pants

  🖼️  Image 1/20: Screenshot 2026-01-26 202551.png
     ⬇️  Downloading from S3...
     ✅ Downloaded (shape: (563, 307, 3))
     🔄 Creating 10 augmented versions...
        ✔️  Created 10/10 augmentations
     ✅ Completed augmentation for Screenshot 2026-01-26 202551.png

  🖼️  Image 2/20: Screenshot 2026-01-26 202629.png
     ⬇️  Downloading from S3...
     ✅ Downloaded (shape: (627, 317, 3))
     🔄 Creating 10 augmented versions...
        ✔️  Created 10/10 augmentations
     ✅ Completed augmentation for Screenshot 2026-01-26 202629.png

  🖼️  Image 3/20: Screenshot 2026-01-26 202650.png
     ⬇️  Downloading from S3...
     ✅ Downloaded (shape: (615, 317, 3))
     🔄 Creating 10 augmented versions...
        ✔️  Created 10/10 augmentations
     ✅ Completed augmentation for Screenshot 2026-01-26 202650.png

  🖼️  Image 4/20: Screenshot 2026-01-26 202708.png
     ⬇️  Downloading fr

In [12]:
# Cell 8: Display final summary
print(f"\n{'='*60}")
print(f"🎉 DATA AUGMENTATION COMPLETE!")
print(f"{'='*60}")
print(f"📊 Final Summary:")
print(f"   - Folders processed: {len(folders)}")
print(f"   - Images per folder: {TARGET_IMAGES_PER_FOLDER}")
print(f"   - Total images created: {total_images_created}")
print(f"   - Expected total: {len(folders) * TARGET_IMAGES_PER_FOLDER}")
print(f"   - Source: s3://{BUCKET}/{SOURCE_PREFIX}")
print(f"   - Target: s3://{BUCKET}/{TARGET_PREFIX}")
print(f"{'='*60}")

# Verify if we hit the target
if total_images_created == len(folders) * TARGET_IMAGES_PER_FOLDER:
    print("✅ Target achieved successfully!")
else:
    print(f"⚠️  Created {total_images_created} images, expected {len(folders) * TARGET_IMAGES_PER_FOLDER}")


🎉 DATA AUGMENTATION COMPLETE!
📊 Final Summary:
   - Folders processed: 5
   - Images per folder: 200
   - Total images created: 1000
   - Expected total: 1000
   - Source: s3://e9tcw5eupu/tight loose clothing model new/data/lower body clothing categories/
   - Target: s3://e9tcw5eupu/tight loose clothing model new augmented/data/lower body clothing categories/
✅ Target achieved successfully!


### Data Augementation for upper body clothing categories

In [13]:
# Cell 1: Import required libraries
import os
import boto3
from botocore.exceptions import ClientError
import albumentations as A
import cv2
import numpy as np
from pathlib import Path
import io

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [14]:
# Cell 2: Define RunPod S3 credentials and paths for UPPER BODY
ACCESS_KEY = "user_37sKcYrvnk9UXaIY3B3Zr90MH0g"
SECRET_KEY = "rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n"
BUCKET = "e9tcw5eupu"
ENDPOINT_URL = "https://s3api-eu-ro-1.runpod.io"
REGION = "eu-ro-1"

# S3 paths for UPPER BODY categories
SOURCE_PREFIX = "tight loose clothing model new/data/upper body clothing categories/"
TARGET_PREFIX = "tight loose clothing model new augmented/data/upper body clothing categories/"

print("✅ Credentials and paths configured for UPPER BODY")
print(f"📂 Source: {SOURCE_PREFIX}")
print(f"📂 Target: {TARGET_PREFIX}")

✅ Credentials and paths configured for UPPER BODY
📂 Source: tight loose clothing model new/data/upper body clothing categories/
📂 Target: tight loose clothing model new augmented/data/upper body clothing categories/


In [15]:
# Cell 3: Initialize S3 client and test connection
print("🔌 Connecting to RunPod S3...")
s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT_URL,
    region_name=REGION,
)

# Test connection
try:
    r = s3.list_objects_v2(Bucket=BUCKET, MaxKeys=3)
    print(f"✅ Connected successfully. KeyCount: {r.get('KeyCount', 0)}")
except ClientError as e:
    print(f"❌ Connection failed: {e}")
    raise

🔌 Connecting to RunPod S3...
✅ Connected successfully. KeyCount: 3


In [16]:
# Cell 4: Define augmentation pipeline
print("🎨 Setting up augmentation pipeline...")

# Create augmentation pipeline with all transformations
augmentation_pipeline = A.Compose([
    # Apply random brightness and contrast changes
    A.RandomBrightnessContrast(
        brightness_limit=0.1,  # Limit brightness change to ±10%
        contrast_limit=0.1,    # Limit contrast change to ±10%
        p=0.8                  # Apply this augmentation 80% of the time
    ),
    # Apply random hue, saturation, and value changes
    A.HueSaturationValue(
        hue_shift_limit=5,     # Shift hue by ±5 degrees
        sat_shift_limit=10,    # Shift saturation by ±10
        val_shift_limit=5,     # Shift value/brightness by ±5
        p=0.6                  # Apply this augmentation 60% of the time
    ),
    # Add random Gaussian noise to the image
    A.GaussNoise(
        var_limit=(5.0, 15.0), # Noise variance between 5 and 15
        p=0.5                  # Apply this augmentation 50% of the time
    ),
    # Apply random Gaussian blur
    A.GaussianBlur(
        blur_limit=(3, 5),     # Kernel size between 3 and 5
        p=0.4                  # Apply this augmentation 40% of the time
    )
])

print("✅ Augmentation pipeline ready with 4 transformations")

🎨 Setting up augmentation pipeline...
✅ Augmentation pipeline ready with 4 transformations


/tmp/ipykernel_2060/3576183489.py:20: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(


In [17]:
# Cell 5: List all folders in UPPER BODY source directory
print(f"📂 Listing folders in: {SOURCE_PREFIX}")

# Query S3 for folders (using Delimiter to get only direct subfolders)
response = s3.list_objects_v2(Bucket=BUCKET, Prefix=SOURCE_PREFIX, Delimiter='/')

# Initialize empty list to store folder names
folders = []

# Extract folder names from common prefixes
if 'CommonPrefixes' in response:
    for prefix in response['CommonPrefixes']:
        folder_path = prefix['Prefix']  # Get full folder path
        folder_name = folder_path.rstrip('/').split('/')[-1]  # Extract just folder name
        folders.append(folder_name)  # Add to list

# Display found folders
print(f"✅ Found {len(folders)} folders:")
for idx, folder in enumerate(folders, 1):
    print(f"   {idx}. {folder}")

📂 Listing folders in: tight loose clothing model new/data/upper body clothing categories/
✅ Found 7 folders:
   1. blouse
   2. crop top
   3. hoodie
   4. shirt
   5. sweater
   6. tank top
   7. tshirt


In [18]:
# Cell 6: Configure augmentation parameters
# Target number of images for each folder
TARGET_IMAGES_PER_FOLDER = 200

# Original number of images in each folder
ORIGINAL_IMAGES_PER_FOLDER = 20

# Calculate how many augmented versions needed per original image
AUGMENTATIONS_PER_IMAGE = TARGET_IMAGES_PER_FOLDER // ORIGINAL_IMAGES_PER_FOLDER

# Display configuration
print(f"📊 Augmentation Configuration for UPPER BODY:")
print(f"   - Original images per folder: {ORIGINAL_IMAGES_PER_FOLDER}")
print(f"   - Target images per folder: {TARGET_IMAGES_PER_FOLDER}")
print(f"   - Augmentations per original image: {AUGMENTATIONS_PER_IMAGE}")
print(f"   - Total folders: {len(folders)}")
print(f"   - Expected total images: {len(folders) * TARGET_IMAGES_PER_FOLDER}")

📊 Augmentation Configuration for UPPER BODY:
   - Original images per folder: 20
   - Target images per folder: 200
   - Augmentations per original image: 10
   - Total folders: 7
   - Expected total images: 1400


In [19]:
# Cell 7: Main augmentation loop for UPPER BODY
# Initialize counter for total images created across all folders
total_images_created = 0

# Loop through each folder
for folder_idx, folder_name in enumerate(folders, 1):
    print(f"\n{'='*60}")
    print(f"📁 Processing folder {folder_idx}/{len(folders)}: {folder_name}")
    print(f"{'='*60}")
    
    # Define source and target paths for current folder
    folder_source_prefix = f"{SOURCE_PREFIX}{folder_name}/"
    folder_target_prefix = f"{TARGET_PREFIX}{folder_name}/"
    
    # List all objects in current source folder
    print(f"🔍 Listing images in {folder_name}...")
    response = s3.list_objects_v2(Bucket=BUCKET, Prefix=folder_source_prefix)
    
    # Initialize list to store image keys (S3 paths)
    image_keys = []
    
    # Filter for image files only
    if 'Contents' in response:
        for obj in response['Contents']:
            key = obj['Key']  # Get S3 object key (path)
            # Skip if it's a folder marker
            if key.endswith('/'):
                continue
            # Check if file has image extension
            if key.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                image_keys.append(key)  # Add to list of image keys
    
    print(f"✅ Found {len(image_keys)} images in {folder_name}\n")
    
    # Initialize counter for images created in current folder
    folder_images_created = 0
    
    # Process each original image in the folder
    for img_idx, image_key in enumerate(image_keys, 1):
        # Extract filename from full S3 path
        original_filename = image_key.split('/')[-1]
        # Split filename into name and extension
        original_name, original_ext = os.path.splitext(original_filename)
        
        print(f"  🖼️  Image {img_idx}/{len(image_keys)}: {original_filename}")
        
        # Download image from S3
        print(f"     ⬇️  Downloading from S3...")
        img_obj = s3.get_object(Bucket=BUCKET, Key=image_key)
        img_data = img_obj['Body'].read()  # Read image bytes
        
        # Convert bytes to numpy array
        img_array = np.frombuffer(img_data, np.uint8)
        # Decode array into OpenCV image format
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        
        # Check if image was decoded successfully
        if image is None:
            print(f"     ❌ Failed to decode image, skipping...")
            continue
        
        print(f"     ✅ Downloaded (shape: {image.shape})")
        
        # Create multiple augmented versions of current image
        print(f"     🔄 Creating {AUGMENTATIONS_PER_IMAGE} augmented versions...")
        for aug_idx in range(1, AUGMENTATIONS_PER_IMAGE + 1):
            # Apply augmentation pipeline to image
            augmented = augmentation_pipeline(image=image)
            augmented_image = augmented['image']  # Get augmented image
            
            # Create new filename with augmentation index
            new_filename = f"{original_name}_aug{aug_idx}{original_ext}"
            # Create full S3 key (path) for new image
            new_key = f"{folder_target_prefix}{new_filename}"
            
            # Encode augmented image back to bytes
            _, buffer = cv2.imencode(original_ext, augmented_image)
            img_bytes = io.BytesIO(buffer.tobytes())
            
            # Upload augmented image to S3
            s3.put_object(
                Bucket=BUCKET,
                Key=new_key,
                Body=img_bytes.getvalue(),
                ContentType='image/jpeg' if original_ext.lower() in ['.jpg', '.jpeg'] else 'image/png'
            )
            
            # Increment counters
            folder_images_created += 1
            total_images_created += 1
            
            # Print progress every 20 images or at the end
            if aug_idx % 20 == 0 or aug_idx == AUGMENTATIONS_PER_IMAGE:
                print(f"        ✔️  Created {aug_idx}/{AUGMENTATIONS_PER_IMAGE} augmentations")
        
        print(f"     ✅ Completed augmentation for {original_filename}\n")
    
    # Print summary for current folder
    print(f"✅ Folder '{folder_name}' complete: {folder_images_created} images created")
    print(f"📊 Total progress: {total_images_created}/{len(folders) * TARGET_IMAGES_PER_FOLDER} images")


📁 Processing folder 1/7: blouse
🔍 Listing images in blouse...
✅ Found 20 images in blouse

  🖼️  Image 1/20: Screenshot 2026-01-23 190627.png
     ⬇️  Downloading from S3...
     ✅ Downloaded (shape: (592, 362, 3))
     🔄 Creating 10 augmented versions...
        ✔️  Created 10/10 augmentations
     ✅ Completed augmentation for Screenshot 2026-01-23 190627.png

  🖼️  Image 2/20: Screenshot 2026-01-23 190652.png
     ⬇️  Downloading from S3...
     ✅ Downloaded (shape: (515, 327, 3))
     🔄 Creating 10 augmented versions...
        ✔️  Created 10/10 augmentations
     ✅ Completed augmentation for Screenshot 2026-01-23 190652.png

  🖼️  Image 3/20: Screenshot 2026-01-23 190819.png
     ⬇️  Downloading from S3...
     ✅ Downloaded (shape: (575, 373, 3))
     🔄 Creating 10 augmented versions...
        ✔️  Created 10/10 augmentations
     ✅ Completed augmentation for Screenshot 2026-01-23 190819.png

  🖼️  Image 4/20: Screenshot 2026-01-23 190837.png
     ⬇️  Downloading from S3...
     ✅

In [20]:
# Cell 8: Display final summary for UPPER BODY
print(f"\n{'='*60}")
print(f"🎉 UPPER BODY DATA AUGMENTATION COMPLETE!")
print(f"{'='*60}")
print(f"📊 Final Summary:")
print(f"   - Folders processed: {len(folders)}")
print(f"   - Images per folder: {TARGET_IMAGES_PER_FOLDER}")
print(f"   - Total images created: {total_images_created}")
print(f"   - Expected total: {len(folders) * TARGET_IMAGES_PER_FOLDER}")
print(f"   - Source: s3://{BUCKET}/{SOURCE_PREFIX}")
print(f"   - Target: s3://{BUCKET}/{TARGET_PREFIX}")
print(f"{'='*60}")

# Verify if we hit the target
if total_images_created == len(folders) * TARGET_IMAGES_PER_FOLDER:
    print("✅ Target achieved successfully!")
else:
    print(f"⚠️  Created {total_images_created} images, expected {len(folders) * TARGET_IMAGES_PER_FOLDER}")


🎉 UPPER BODY DATA AUGMENTATION COMPLETE!
📊 Final Summary:
   - Folders processed: 7
   - Images per folder: 200
   - Total images created: 1400
   - Expected total: 1400
   - Source: s3://e9tcw5eupu/tight loose clothing model new/data/upper body clothing categories/
   - Target: s3://e9tcw5eupu/tight loose clothing model new augmented/data/upper body clothing categories/
✅ Target achieved successfully!
